In [1]:
import pandas as pd
import numpy as np
import talib
from typing import List
import akshare as ak
from tabulate import tabulate

Instruments = [
    "JM0",
    "RB0",
    "SM0",
    "SA0",
    "FG0",
    "UR0",
    "MA0",
    "AG0",
    "TA0",
    "SP0",
    "V0",
    "P0",
    "Y0",
    "RM0",
    "FU0",
    "SR0",
]

def getOperateDirection(histPrice, stage: int):
    """
    获取当前均线方向
    """
    MAX = talib.MA(histPrice["close"], stage)
    MAX_2 = talib.MA(histPrice["close"], stage // 2)
    MAX_4 = talib.MA(histPrice["close"], stage // 4)
    direct = 0
    maxGapPercentage: float = 0.02
    if MAX_4.iloc[-2] > MAX.iloc[-2] and MAX_2.iloc[-2] > MAX.iloc[-2]:
        direct = 1
    elif MAX_4.iloc[-2] < MAX.iloc[-2] and MAX_2.iloc[-2] < MAX.iloc[-2]:
        direct = -1
    else:
        gapPercentage = abs((MAX_4.iloc[-2] - MAX.iloc[-2]) / MAX.iloc[-2])
        if gapPercentage < maxGapPercentage:
            return 1 if MAX_2.iloc[-2] > MAX.iloc[-2] else -1
    return direct

def MACD(data, fast_period=10, slow_period=20, singal_period=5):
    fastMA = talib.MA(data, fast_period)
    slowMA = talib.MA(data, slow_period)
    macdLine: List[float] = np.array(fastMA, dtype=float) - np.array(slowMA, dtype=float)
    signalLine = talib.MA(macdLine, singal_period)
    macdList = macdLine - np.array(signalLine)
    return macdLine, signalLine, macdList

def checkMACDCross(histPrice, stage: int):
    """
    crossType为1表示金叉，为-1表示死叉，为0表示没有
    diffZeroAxis为1表示在DIFF>10时出现的，为-1表示在DIFF<-10出现的，为0表示在[-10,10]区间出现的
    """
    # DIFF, DEA, _MACD = talib.MACD(histPrice["Close"], stage // 2, stage, stage // 4)
    DIFF, DEA, _MACD = MACD(histPrice["close"], stage // 2, stage, stage // 4)
    if len(DIFF) < 2 or len(DEA) < 2:
        return (0, 0)
    latest_DIFF = DIFF[-1]
    previous_DIFF = DIFF[-2]
    latest_DEA = DEA[-1]
    previous_DEA = DEA[-2]
    crossType = 0
    diffZeroAxis = 0
    # 金叉条件
    if latest_DIFF > latest_DEA and previous_DIFF <= previous_DEA:
        crossType = 1
    # 死叉条件
    elif latest_DIFF < latest_DEA and previous_DIFF >= previous_DEA:
        crossType = -1
    else:
        crossType = 0
    if abs(latest_DIFF) <= 10:
        diffZeroAxis = 0
    else:
        if latest_DIFF < 0:
            diffZeroAxis = -1
        else:
            diffZeroAxis = 1
    return (crossType, diffZeroAxis)
        
class FutureManager:
    symbol = ''
    analysisDate = ""
    direct_20 = 0
    direct_80 = 0
    direct_320 = 0
    closePrice = 0
    a0CrossType = 0
    a0DiffZeroAxis = False
    def __init__(self, symbol):
        self.symbol = symbol

    def analysis(self):
        # 15min一根K线构成的数据
        histPrice15 = ak.futures_zh_minute_sina(symbol=self.symbol, period="15")
        self.analysisBase(histPrice15)
        self.analysisDirect(histPrice15)
        self.analysisMACD(histPrice15)

    def analysisBase(self, histPrice):
        self.closePrice = histPrice.iloc[-1]["close"]
        self.analysisDate = histPrice.iloc[-1]["datetime"]

    def analysisDirect(self, histPrice):
        self.direct_20 = getOperateDirection(histPrice, 20)
        self.direct_80 = getOperateDirection(histPrice, 80)
        self.direct_320 = getOperateDirection(histPrice, 320)
    
    def analysisMACD(self, histPrice):
        crossType, diffZeroAxis = checkMACDCross(histPrice, 20)
        self.a0CrossType = crossType
        self.a0DiffZeroAxis = diffZeroAxis

def operateWarn(direct_20: int, direct_80: int, direct_320: int) -> int:
    if direct_80 == -1 and direct_320 == -1:
        if direct_20 == 1:
            return -2
        else:
            return -1
    elif direct_80 == 1 and direct_320 == 1:
        if direct_20 == -1:
            return 2
        else:
            return 1
    else:
        return 0

def macdCrossWarn(corssType: int, diffZeroAxis: int):
    if corssType == 1:
        if diffZeroAxis == -1:
            return 3
        elif diffZeroAxis == 1:
            return 2
        else:
            return 1
    elif corssType == -1:
        if diffZeroAxis == 1:
            return -3
        elif diffZeroAxis == -1:
            return -2
        else:
            return -1
    else:
        return 0

def onTick():
    trendMap = {
        0: "行情模糊",
        1: "上涨",
        -1: "下跌",
    }
    operateMap = {
        3: "\033[31m 15min和1h、4h趋势相反，出现金叉信号，开仓做多 \033[0m",
        2: "15min和1h、4h趋势相反，找机会做多",
        1: "1h、4h大趋势方向一致，持仓或等15min级别回调做多",
        0: "\033[33m 1h、4h趋势不一致，注意行情变盘 \033[0m",
        -1: "1h、4h大趋势方向一致，持仓或等15min级别回调做空",
        -2: "15min和1h、4h趋势相反，找机会做空",
        -3: "\033[31m 15min和1h、4h趋势相反，出现死叉信号，开仓做空 \033[0m",
    }
    crossMap = {
        3: "金叉",
        2: "零轴上的金叉，上涨趋势很可能会延续",
        1: "弱势金叉",
        0: "-",
        -1: "弱势死叉",
        -2: "零轴下的死叉，下跌趋势很可能会延续",
        -3: "死叉",
    }
    outputDf = pd.DataFrame([], columns=["合约主连代码", "执行时间", "15min方向", "1h方向", "4h方向", "MACD金死叉", "操作"])
    for symbol in Instruments:
        fObj = FutureManager(symbol)
        fObj.analysis()
        warnNum = operateWarn(fObj.direct_20, fObj.direct_80, fObj.direct_320)
        crossNum = macdCrossWarn(fObj.a0CrossType, fObj.a0DiffZeroAxis)
        outputDf.loc[len(outputDf.index)] = [
            symbol,
            fObj.analysisDate,
            trendMap[fObj.direct_20],
            trendMap[fObj.direct_80],
            trendMap[fObj.direct_320],
            crossMap[crossNum],
            operateMap[warnNum],
        ]
    print(tabulate(outputDf, headers="keys", tablefmt="fancy_outline", numalign="center"))

if __name__ == "__main__":
    onTick()

╒════╤════════════════╤═════════════════════╤═════════════╤══════════╤══════════╤══════════════╤═════════════════════════════════════════════════╕
│    │ 合约主连代码   │ 执行时间            │ 15min方向   │ 1h方向   │ 4h方向   │ MACD金死叉   │ 操作                                            │
╞════╪════════════════╪═════════════════════╪═════════════╪══════════╪══════════╪══════════════╪═════════════════════════════════════════════════╡
│ 0  │ JM0            │ 2024-08-02 23:00:00 │ 上涨        │ 下跌     │ 下跌     │ -            │ 15min和1h、4h趋势相反，找机会做空               │
│ 1  │ RB0            │ 2024-08-02 23:00:00 │ 上涨        │ 上涨     │ 下跌     │ -            │  1h、4h趋势不一致，注意行情变盘                 │
│ 2  │ SM0            │ 2024-08-02 15:00:00 │ 上涨        │ 下跌     │ 下跌     │ -            │ 15min和1h、4h趋势相反，找机会做空               │
│ 3  │ SA0            │ 2024-08-02 23:00:00 │ 上涨        │ 上涨     │ 下跌     │ -            │  1h、4h趋势不一致，注意行情变盘                 │
│ 4  │ FG0            │ 2024-08-02 23:00:00 │ 下跌        │ 下跌     │